# 参考：Pytorch 之 MNIST 数据集实现
  
 链接🔗：https://blog.csdn.net/weixin_37589575/article/details/95856667

In [1]:
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os
import torchvision
import numpy as np
from torch.autograd import Variable
%matplotlib inline

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize(mean = [0.5, 0.5 ,0.5], std = [0.5, 0.5, 0.5])])
# 图片变换:
# ToTensor()把灰度范围从0-255变换到0-1之间
# transform.Normalize()把0-1变换到(-1,1)

In [10]:
data_train = datasets.MNIST(root = "./data/",
                            transform = transform,
                            train = True,
                            download = True)

data_test = datasets.MNIST(root="./data/",
                           transform = transform,
                           train = False)

In [11]:
data_loader_train = torch.utils.data.DataLoader(dataset = data_train,
                                               batch_size = 64,
                                               shuffle = True,
                                               num_workers = 2)
data_loader_test = torch.utils.data.DataLoader(dataset = data_test,
                                              batch_size = 64,
                                              shuffle = True,
                                              num_workers = 2)
'''
dataset: 将要加载的数据的数据集
batch_size(int): 每迭代加载的图像的数量
shuffle(bool):将其设置为True时，数据将会在每个epoch之后，进行打乱重新排序
num_workers(int):定义加在数据的线程数，默认为0，意味着只使用主线程加载数据。
'''

'\ndataset: 将要加载的数据的数据集\nbatch_size(int): 每迭代加载的图像的数量\nshuffle(bool):将其设置为True时，数据将会在每个epoch之后，进行打乱重新排序\nnum_workers(int):定义加在数据的线程数，默认为0，意味着只使用主线程加载数据。\n'

In [12]:
print(len(data_train))

60000


In [13]:
print(len(data_test))

10000


In [8]:
print(data_train[0])  # 会报错

RuntimeError: output with shape [1, 28, 28] doesn't match the broadcast shape [3, 28, 28]

In [15]:
images, labels = next(iter(data_loader_train))
print(imagesimg = torchvision.utils.make_grid(images)

img = img.numpy().transpose(1,2,0)
std = [0.5,0.5,0.5]
mean = [0.5,0.5,0.5]
img = img*std+mean
print([labels[i] for i in range(64)])
plt.imshow(img)

SyntaxError: invalid syntax (<ipython-input-15-3f560bd4bb66>, line 4)

## 重新开始

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [18]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break

In [19]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [3]:
seed = 1
batch_size = 64

In [4]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'batch_size': batch_size}

if use_cuda:
    kwargs.update({'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True},
                   )

transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))])

In [5]:
learning_rate = 0.008
epochs = 20
gamma = 0.1
log_interval = 2

In [6]:
dataset1 = datasets.MNIST('./data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('./data', train=False, transform=transform)

In [7]:
train_loader = torch.utils.data.DataLoader(dataset1,**kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

In [8]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma = gamma)

In [9]:
for epoch in range(1, epochs + 1):
    # train(args, model, device, train_loader, optimizer, epoch)
    print(epoch)
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 32 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            # if args.dry_run:
            #     break
    
    # test(model, device, test_loader)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    scheduler.step()


1
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.335568
Train Epoch: 1 [2048/60000 (3%)]	Loss: 2.152317
Train Epoch: 1 [4096/60000 (7%)]	Loss: 1.990494
Train Epoch: 1 [6144/60000 (10%)]	Loss: 1.736615
Train Epoch: 1 [8192/60000 (14%)]	Loss: 1.368946
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.263340
Train Epoch: 1 [12288/60000 (20%)]	Loss: 0.962070
Train Epoch: 1 [14336/60000 (24%)]	Loss: 0.882994
Train Epoch: 1 [16384/60000 (27%)]	Loss: 1.013182
Train Epoch: 1 [18432/60000 (31%)]	Loss: 0.959295
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.849990
Train Epoch: 1 [22528/60000 (38%)]	Loss: 0.623258
Train Epoch: 1 [24576/60000 (41%)]	Loss: 0.977802
Train Epoch: 1 [26624/60000 (44%)]	Loss: 0.674462
Train Epoch: 1 [28672/60000 (48%)]	Loss: 0.822928
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.413114
Train Epoch: 1 [32768/60000 (55%)]	Loss: 0.841556
Train Epoch: 1 [34816/60000 (58%)]	Loss: 0.630331
Train Epoch: 1 [36864/60000 (61%)]	Loss: 0.478038
Train Epoch: 1 [38912/60000 (65%)]	Loss: 0.473770
Train Epo

Train Epoch: 6 [18432/60000 (31%)]	Loss: 0.393795
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.533592
Train Epoch: 6 [22528/60000 (38%)]	Loss: 0.490829
Train Epoch: 6 [24576/60000 (41%)]	Loss: 0.463020
Train Epoch: 6 [26624/60000 (44%)]	Loss: 0.578163
Train Epoch: 6 [28672/60000 (48%)]	Loss: 0.305690
Train Epoch: 6 [30720/60000 (51%)]	Loss: 0.683618
Train Epoch: 6 [32768/60000 (55%)]	Loss: 0.344311
Train Epoch: 6 [34816/60000 (58%)]	Loss: 0.237790
Train Epoch: 6 [36864/60000 (61%)]	Loss: 0.327802
Train Epoch: 6 [38912/60000 (65%)]	Loss: 0.437296
Train Epoch: 6 [40960/60000 (68%)]	Loss: 0.367026
Train Epoch: 6 [43008/60000 (72%)]	Loss: 0.483164
Train Epoch: 6 [45056/60000 (75%)]	Loss: 0.276051
Train Epoch: 6 [47104/60000 (78%)]	Loss: 0.630269
Train Epoch: 6 [49152/60000 (82%)]	Loss: 0.667350
Train Epoch: 6 [51200/60000 (85%)]	Loss: 0.423695
Train Epoch: 6 [53248/60000 (89%)]	Loss: 0.562194
Train Epoch: 6 [55296/60000 (92%)]	Loss: 0.609625
Train Epoch: 6 [57344/60000 (96%)]	Loss: 0.569428


Train Epoch: 11 [34816/60000 (58%)]	Loss: 0.404816
Train Epoch: 11 [36864/60000 (61%)]	Loss: 0.544327
Train Epoch: 11 [38912/60000 (65%)]	Loss: 0.323941
Train Epoch: 11 [40960/60000 (68%)]	Loss: 0.337964
Train Epoch: 11 [43008/60000 (72%)]	Loss: 0.472959
Train Epoch: 11 [45056/60000 (75%)]	Loss: 0.518573
Train Epoch: 11 [47104/60000 (78%)]	Loss: 0.604463
Train Epoch: 11 [49152/60000 (82%)]	Loss: 0.392914
Train Epoch: 11 [51200/60000 (85%)]	Loss: 0.441741
Train Epoch: 11 [53248/60000 (89%)]	Loss: 0.274751
Train Epoch: 11 [55296/60000 (92%)]	Loss: 0.398269
Train Epoch: 11 [57344/60000 (96%)]	Loss: 0.599217
Train Epoch: 11 [59392/60000 (99%)]	Loss: 0.431373

Test set: Average loss: 0.2926, Accuracy: 9196/10000 (92%)

12
Train Epoch: 12 [0/60000 (0%)]	Loss: 0.212117
Train Epoch: 12 [2048/60000 (3%)]	Loss: 0.476271
Train Epoch: 12 [4096/60000 (7%)]	Loss: 0.317407
Train Epoch: 12 [6144/60000 (10%)]	Loss: 0.376307
Train Epoch: 12 [8192/60000 (14%)]	Loss: 0.496690
Train Epoch: 12 [10240/60000 

Train Epoch: 16 [47104/60000 (78%)]	Loss: 0.432622
Train Epoch: 16 [49152/60000 (82%)]	Loss: 0.480397
Train Epoch: 16 [51200/60000 (85%)]	Loss: 0.514156
Train Epoch: 16 [53248/60000 (89%)]	Loss: 0.725302
Train Epoch: 16 [55296/60000 (92%)]	Loss: 0.240265
Train Epoch: 16 [57344/60000 (96%)]	Loss: 0.385697
Train Epoch: 16 [59392/60000 (99%)]	Loss: 0.675486

Test set: Average loss: 0.2926, Accuracy: 9196/10000 (92%)

17
Train Epoch: 17 [0/60000 (0%)]	Loss: 0.369732
Train Epoch: 17 [2048/60000 (3%)]	Loss: 0.439989
Train Epoch: 17 [4096/60000 (7%)]	Loss: 0.425520
Train Epoch: 17 [6144/60000 (10%)]	Loss: 0.533557
Train Epoch: 17 [8192/60000 (14%)]	Loss: 0.286752
Train Epoch: 17 [10240/60000 (17%)]	Loss: 0.373064
Train Epoch: 17 [12288/60000 (20%)]	Loss: 0.377319
Train Epoch: 17 [14336/60000 (24%)]	Loss: 0.488367
Train Epoch: 17 [16384/60000 (27%)]	Loss: 0.326482
Train Epoch: 17 [18432/60000 (31%)]	Loss: 0.454120
Train Epoch: 17 [20480/60000 (34%)]	Loss: 0.195377
Train Epoch: 17 [22528/60000 

In [ ]:
torch.save(model.state_dict(), "mnist_cnn.pt")